In [3]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def hopkins_statistic(data, sample_size=50, random_state=None):
    """
    Oblicza statystykę Hopkinsa dla danego zbioru danych.
    
    Parameters:
        data (numpy.ndarray or pandas.DataFrame): Dane do analizy.
        sample_size (int): Liczba próbek do wylosowania.
        random_state (int or None): Losowe ziarno (opcjonalne).
        
    Returns:
        float: Wartość statystyki Hopkinsa (0 - dane losowe, 1 - dane klastrowe).
    """
    if isinstance(data, np.ndarray):
        X = data
    else:
        X = data.to_numpy()
    
    if random_state is not None:
        np.random.seed(random_state)
    
    n, d = X.shape
    if sample_size > n:
        raise ValueError("Sample size cannot exceed number of data points.")
    
    # Losowanie próbek z danych
    indices = np.random.choice(n, sample_size, replace=False)
    samples = X[indices]
    
    # Generowanie losowych punktów w tym samym zakresie
    random_points = np.random.uniform(
        np.min(X, axis=0), np.max(X, axis=0), (sample_size, d)
    )
    
    # Dopasowanie modelu Nearest Neighbors
    nbrs = NearestNeighbors(n_neighbors=2).fit(X)
    
    # Odległości dla próbek z danych
    distances_real, _ = nbrs.kneighbors(samples)
    W = np.sum(distances_real[:, 1])  # 2 najbliższy punkt
    
    # Odległości dla losowych punktów
    distances_random, _ = nbrs.kneighbors(random_points)
    U = np.sum(distances_random[:, 0])  # 1 najbliższy punkt
    
    # Statystyka Hopkinsa
    H = W / (W + U)
    return H


In [ ]:
from sklearn.preprocessing import MinMaxScaler
#import numpy as np
#import pandas as pd

rfm_data = pd.read_csv('analiza_rfm.csv')

# Załóżmy, że analiza_rfm zawiera kolumny 'Recency', 'Frequency', 'Monetary'

# 1. Normalizacja danych
scaler = MinMaxScaler()
rfm_normalized = scaler.fit_transform(rfm_data)

# 2. Obliczenie statystyki Hopkinsa
H = hopkins_statistic(rfm_normalized, sample_size=50, random_state=42)  # Użyj wcześniej zaimplementowanej funkcji
print(f"Statystyka Hopkinsa: {H}")

# Interpretacja wyniku
if H > 0.75:
    print("Dane są wysoce skupiskowe, nadają się do klastrowania.")
elif 0.5 < H <= 0.75:
    print("Dane mogą mieć pewną strukturę skupiskową.")
else:
    print("Dane są rozproszone, klastrowanie może być mniej efektywne.")


Statystyka Hopkinsa: 0.011788158608951263
Dane są rozproszone, klastrowanie może być mniej efektywne.
